In [ ]:
!pip -q install pyicloud
!python3 -m pip -q install numbers-parser
!pip -q install pygsheets
!pip -q install roman
!sudo apt-get -q install libmagic1
!pip -q install tabula-py
!pip -q install pikepdf
!pip -q install -U ExtractTable

Reading package lists...
Building dependency tree...
Reading state information...
libmagic1 is already the newest version (1:5.32-2ubuntu0.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
from shutil import copyfileobj
from numbers_parser import Document
from pyicloud import PyiCloudService
from getpass import getpass
from datetime import datetime

import pandas as pd
from pandas import DataFrame
import numpy as np
import glob
import os
import re
import datetime as dt
import gspread
import tabula
import pikepdf
import gspread_dataframe as gd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

from gspread_dataframe import set_with_dataframe

from google.auth import default
creds, _ = default()

# Authenticate GDrive -- this will ask for two authentication codes
auth.authenticate_user()
gauth = GoogleAuth()
gdrive = GoogleDrive(gauth)
gauth.credentials = GoogleCredentials.get_application_default()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Use this as our Master List of Categories for Credit Cards

In [ ]:
# feel free to customize this, just follow the format

food = ['STARBUCKS', 'SALADSTOP', 'GELATISSIMO', 'FRANKIES', 'WILD FLOUR', 'ONO POKE', 'HALAL', 'CHILIS', 'CBTL', 'JOLLIBEE',
        'GADC', 'YABU', 'KFC', 'FOWL BREAD', 'RED BRICK HOUSE', 'ORDER READY', 'AMICI', '448 HALANG', 'FOOD PANDA',
        'DIN TAI FUNG', 'ANTIPODEAN', 'IPPUDO', 'TOKYO MILK CHEESE', 'CINNABON', 'MARY GRACE', 'BONCHON', 'MOTORINO',
        'CHOWKING', 'IKKOUSHA RAMEN', 'SSP-MACTAN CEBU', 'SEATTLE BEST', 'SEATTLE BEST COFFEE', 'PIZZULU', 'CIBO',
        'FAT SEED', 'ANGKAN', 'NONOS', 'MADMARKS', 'PARADISE DYNASTY', 'KENNY ROGERS', 'TSUJIRI', 'REAL AMERICAN DOUG',
        'YELLOW CAB', 'LOBBY BAKESHOP', '% BGC', 'KALLEFJORD', 'ELEPHANT GROUNDS', 'PINKBERRY', 'COCO ICHIBANYA', 'SUSHI NORI',
        'MCDO','PINKS HOTDOGS', 'WILDFLOUR', 'MEISTER\'S UNCORKED', 'DEAN AND DELUCA'
        ]
travel = ['PETRON', 'SKYWAY', 'BURGER KING', 'EUROFUEL', 'CEBU PAC', 'SHELL', 'QANTAS', 'PAL AIR']
in_app = ['APPLE', 'DISCORD', 'EPIDEMIC SOUND', 'GRAB', 'TUBEBUDDY', 'YOUTUBEPREMIUM', 'GUM.CO/CC*', 'CIBI INFORMATION',
          'SINGAPORE, SGP', 'COGNOSPHERE PTE.', 'SINGAPORESGP', 'SINGAPORE SGP']
entertainment = ['STEAM', 'SM CINEMA', 'SHOP ANYWHERE', 'SPOTIFY', 'VLOUNGE', 'STUDIO 300']
accommodation = ['AIRBNB', 'SHANGRILA', 'SHANGRI-LA', 'AGODA', 'SEDA', 'LAKE HOTEL', 'SEQUOIA', 'HUE PUERTO PRINCESA']
shopping = ['SM STORE', 'FULLY BOOKED', 'FACE SHOP', 'RUNIT', 'FAMILYMART', 'LUSH', 'UNIQLO', 'POWER MAC', 'LEGION', 'H&M', 'INNISFREE',
            'IKEA', 'STRAIGHTFORWARD', 'NIKE', 'MERCURY DRUG', 'TERRY RETAIL', 'DIGITAL WALKER', 'CULLENWAYNE', 'LAZADA', 'POWERMAC', 'SHOPEE',
            'THE LOOP', 'ACE HARDWARE', 'OFFICE WAREHOUSE']
learning = ['COURSERA', 'CODE WITH MOSH', 'UDEMY', 'COURSRA', 'QWIKLABS']
utilities = ['GLOBE', 'SMART', 'PRU LIFE', 'BLOCK SENDER', 'GODADDY', 'BAYAD CENTER']
grocery = ['S&R', 'WALTERMART', 'PUREGOLD', 'WATSONS', 'THE MARKETPLACE', 'THEMARKETPLACE', 'THE MARKET PLACE', 'THEMARKET PLACE']

# merged = food + travel + in_app + entertainment + accommodation + shopping + learning


# Security Bank Credit Card Transactions

In [ ]:
all_files = glob.glob(os.path.join("/content/drive/MyDrive/Personal/Finances/Security Bank Credit Card Transaction History/*.csv")) #edit this directory to your own Security Bank CSVs

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=False, header=1)
    li.append(df)

df = pd.concat(li, axis=0)

In [ ]:
rename_cols = {'Posting Date"': 'posting_date', 'Transaction Date': 'transaction_date', 
               'Establishment': 'establishment', 'Currency': 'currency', 'Debit': 'debit',
               'Credit': 'credit'}

df.rename(columns=rename_cols, inplace=True)

In [ ]:
df.sort_values(by='transaction_date', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# drop duplicate rows for the combined sbc tables

df.drop_duplicates(subset=['posting_date', 'transaction_date', 'establishment', 'debit', 'credit'], inplace=True)

In [ ]:
df['establishment'] = df['establishment'].str.strip()
df['establishment'] = df['establishment'].str.upper()
df['category'] = np.nan

In [ ]:
for item in travel:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Travel'

for item in food:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Food'

for item in in_app:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'In-App Purchase'

for item in entertainment:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Entertainment'

for item in accommodation:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Accommodation'

for item in shopping:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Shopping'

for item in learning:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Learning'

for item in utilities:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Utilities'

for item in grocery:
  df.loc[df['establishment'].str.contains(item, regex=True), 'category'] = 'Grocery'

df['category'].fillna('Others', inplace=True)

In [ ]:
df.dropna(subset=['debit'], inplace=True)

In [ ]:
rename_columns = {'transaction_date': 'Date', 'establishment': 'Item', 'debit': 'Price', 'category': 'Category'}
df.rename(columns=rename_columns, inplace=True)
df['Price'] = df['Price'].apply(lambda x: re.sub(',', '', x))
df['Price'] = df['Price'].astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Payment Type'] = 'Security Bank Platinum Credit Card'
df['Date'] = df['Date'].apply(lambda x: x.date())
final_df_sbc = df[['Date', 'Price', 'Item', 'Category', 'Payment Type']]

In [ ]:
final_df_sbc[final_df_sbc['Category'] == 'Others']

,Date,Price,Item,Category,Payment Type
43,2021-12-16,2500.00,ANNUAL MEMBERSHIP FEE PRIMARY,Others,Security Bank Platinum Credit Card
251,2022-03-23,927.00,SM PRIME ECOM,Others,Security Bank Platinum Credit Card
269,2022-03-28,382.00,283 FORBESTOWN CENTER,Others,Security Bank Platinum Credit Card
307,2022-04-03,309.00,SM PRIME ECOM,Others,Security Bank Platinum Credit Card
363,2022-04-13,480.94,OF,Others,Security Bank Platinum Credit Card
445,2022-04-25,6064.80,VITABLE,Others,Security Bank Platinum Credit Card
481,2022-05-04,268.48,OF,Others,Security Bank Platinum Credit Card
501,2022-05-15,536.93,OF,Others,Security Bank Platinum Credit Card
525,2022-05-21,535.57,OF,Others,Security Bank Platinum Credit Card
551,2022-06-08,260.00,TMBP,Others,Security Bank Platinum Credit Card


#Union Bank x Lazada Credit Card Transactions 

In [ ]:
gc = gspread.authorize(creds)

# what I do is that I copy-paste the UB x Lazada SOAs excel files into just 1 Google Sheets, then this script will clean that combined Google Sheet

wb = gc.open_by_url('PUT_GOOGLE_SHEETS_LINK_HERE') 
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE_DIRTY'], format='%m/%d/%Y')

In [ ]:
df.sort_values(by='DATE', inplace=True)

In [ ]:
rename_cols = {'DATE_DIRTY': 'Date_Dirty', 'DATE': 'Date', 'DESCRIPTION': 'Item', 'AMOUNT': 'Price'}
df.rename(columns=rename_cols, inplace=True)
df['Payment Type'] = 'UnionBank x Lazada Credit Card'
df['Category'] = np.nan
df['Item'] = df['Item'].str.upper()
df['Price'] = df['Price'].str.replace("PHP ", "")
df['Price'] = df['Price'].str.replace(",", "")
df['Price'] = df['Price'].astype(float)

In [ ]:
for item in travel:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Travel'

for item in food:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Food'

for item in in_app:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'In-App Purchase'

for item in entertainment:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Entertainment'

for item in accommodation:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Accommodation'

for item in shopping:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Shopping'

for item in learning:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Learning'

for item in utilities:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Utilities'

for item in grocery:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Grocery'

df['Category'].fillna('Others', inplace=True)

In [ ]:
# remove all payment rows
df = df[~df['Item'].isin(['PHONE PAYMENT', 'BANCNET PAYMENT'])]
df['Date'] = df['Date'].apply(lambda x: x.date())
final_df_ub = df[['Date', 'Price', 'Item', 'Category', 'Payment Type']]

<ipython-input-43-0ba0e67280d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#EastWest Bank Credit Card Transactions

## Unfortunately, not all PDF SOAs can be read by tabula-py

In the case that the table cannot be detected by tabula-py, what you can do is to take a screenshot of the SOA from PDF, upload to [ExtractTable](https://extracttable.com/), copy the table extracted from there, and then paste to our Google Sheets.

Take note that you might need to do some additional cleaning on some SOAs

In [ ]:
pdf = pikepdf.open('PUT_EW_BANK_SOA', password='PUT_PASSWORD')
pdf.save('extractable.pdf')

In [ ]:
df = tabula.read_pdf("/content/extractable.pdf", pages='all')

Dec 28, 2022 8:03:24 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 28, 2022 8:03:29 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 28, 2022 8:03:30 PM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Dec 28, 2022 8:03:31 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
df[1]

,NOV 28,NOV 29,THE MARKETPLACECENT,PHILIPPINES PHL,Unnamed: 0,Unnamed: 1,331.00
0,NaN,NaN,NON-BASIC TRANSACTIONS,NaN,NaN,NaN,NaN
1,NOV 10,NOV 15,INTERNET PAYMENT,NaN,NaN,NaN,"-7,030.35"
2,NOV 13,NOV 14,COURSRA6QY2ZVYAY05KK5,6509639884USA,NaN,USD 14.00,814.98
3,NOV 16,NOV 17,CRO TOPUP INTERNET GFS S INTERNET,SGP,NaN,SGD 20.20,862.66
4,NOV 18,NOV 21,JOLLIBEE ONLINEPASIG,PHL,NaN,NaN,526.00
5,NOV 24,NOV 25,APPLE.COM/BILLITUNES.COM,IRL,NaN,NaN,202.38
6,NOV 24,NOV 25,APPLE.COM/BILLITUNES.COM,IRL,NaN,NaN,405.78
7,NOV 26,NOV 28,GRABMAKATI,PHL,NaN,NaN,412.00
8,NOV 27,NOV 28,GRABMAKATI,PHL,NaN,NaN,545.00
9,NOV 27,NOV 29,STEAM PURCHASESEATTLE,DEU,NaN,NaN,137.27


In [ ]:
df[1].iloc[0] = df[1].columns
list_rename = {0: 'Transaction Date', 1: 'Post Date', 2: 'Item', 3: 'Currency', 4: 'Price'}

In [ ]:
for key, value in list_rename.items():
  df[1].rename(columns = {list(df[1])[key]: value}, inplace=True)

In [ ]:
df[1].fillna('', inplace=True)
df[1]['Item'] = df[1]['Item'] + ' ' + df[1]['Currency']
df[1].drop(columns=['Price', 'Currency'], inplace=True)
df[1].rename(columns={'Unnamed: 1': 'Currency'}, inplace=True)

In [ ]:
df[1]

,Transaction Date,Post Date,Item,Currency,331.00
0,NOV 28,NOV 29,THE MARKETPLACECENT PHILIPPINES PHL,Unnamed: 1,331.00
1,NOV 10,NOV 15,INTERNET PAYMENT,,"-7,030.35"
2,NOV 13,NOV 14,COURSRA6QY2ZVYAY05KK5 6509639884USA,USD 14.00,814.98
3,NOV 16,NOV 17,CRO TOPUP INTERNET GFS S INTERNET SGP,SGD 20.20,862.66
4,NOV 18,NOV 21,JOLLIBEE ONLINEPASIG PHL,,526.00
5,NOV 24,NOV 25,APPLE.COM/BILLITUNES.COM IRL,,202.38
6,NOV 24,NOV 25,APPLE.COM/BILLITUNES.COM IRL,,405.78
7,NOV 26,NOV 28,GRABMAKATI PHL,,412.00
8,NOV 27,NOV 28,GRABMAKATI PHL,,545.00
9,NOV 27,NOV 29,STEAM PURCHASESEATTLE DEU,,137.27


In [ ]:
# Export to Google Sheets the FIRST sheet. Can run this only once. No need to run this again. Uncomment this when running the first time

# workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0/')
# worksheet = workbook.sheet1
# worksheet.update([df[1].columns.values.tolist()] + df[1].values.tolist())

In [ ]:
# Code to append rows for succeeding months. Just run once too.

workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0/')
worksheet = workbook.sheet1
worksheet.append_rows(values=[df[1].columns.values.tolist()] + df[1].values.tolist(), insert_data_option='INSERT_ROWS')

{'spreadsheetId': '1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0',
 'tableRange': 'Sheet1!A1:F89',
 'updates': {'spreadsheetId': '1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0',
  'updatedRange': 'Sheet1!A90:E115',
  'updatedRows': 26,
  'updatedColumns': 5,
  'updatedCells': 130}}

## Use this section to clean SOA copy-pasted from ExtractTable.com

In [ ]:
wb = gc.open_by_url('put_google_sheets_link_copied_from_extract_table')
sheet = wb.worksheet('Sheet2')
data = sheet.get_all_values()

df = pd.DataFrame(data)

In [ ]:
df['Transaction Date'] = df[0].str.extract(r'^([\w]+ \d\d)')

In [ ]:
df['Post Date'] = df[0].str.replace(r'^([\w]+ \d\d )', '')

<ipython-input-92-b346d8bd2ff8>:1: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
df.drop(columns=[0,1], inplace=True)

In [ ]:
rename_columns = {2:'Item', 3: 'Currency', 4:'Price'}
df.rename(columns=rename_columns, inplace=True)

In [ ]:
df = df[['Transaction Date', 'Post Date', 'Item', 'Currency', 'Price']]

In [ ]:
# Code to append rows for succeeding months.

workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0/')
worksheet = workbook.sheet1
worksheet.append_rows(values=[df.columns.values.tolist()] + df.values.tolist(), insert_data_option='INSERT_ROWS')

{'spreadsheetId': '1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0',
 'tableRange': 'Sheet1!A1:E42',
 'updates': {'spreadsheetId': '1Gig3RV2hL4iry40QdcsjLwpNMxLah98gDApdTdR94-0',
  'updatedRange': 'Sheet1!A43:E66',
  'updatedRows': 24,
  'updatedColumns': 5,
  'updatedCells': 120}}

##Once done cleaning SOAs, use this section categorize and clean the merged SOA table

In [ ]:
wb = gc.open_by_url('put_merged_SOA')
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

In [ ]:
df['Category'] = np.nan

In [ ]:
for item in travel:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Travel'

for item in food:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Food'

for item in in_app:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'In-App Purchase'

for item in entertainment:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Entertainment'

for item in accommodation:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Accommodation'

for item in shopping:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Shopping'

for item in learning:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Learning'

for item in utilities:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Utilities'

for item in grocery:
  df.loc[df['Item'].str.contains(item, regex=True), 'Category'] = 'Grocery'

df['Category'].fillna('Others', inplace=True)

In [ ]:
df['Date'] = df['Transaction Date'] + ', ' + df['Year']
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y').dt.date
df['Payment Type'] = 'EastWest Gold Visa Credit Card'

In [ ]:
df['Price'] = df['Price'].str.replace(',', '')
df['Price'] = df['Price'].astype(float)

In [ ]:
df = df[df['Price'] > 0]

In [ ]:
final_df_ew = df[['Date', 'Price', 'Item', 'Category', 'Payment Type']]

#Cash Transactions

In [ ]:
# api to access icloud Drive

api = PyiCloudService(getpass('Username:'), getpass('Password:'))

if api.requires_2fa:
    print ("Two-factor authentication required.")
    code = input("Enter the code you received of one of your approved devices: ")
    result = api.validate_2fa_code(code)
    print("Code validation result: %s" % result)

    if not result:
        print("Failed to verify security code")
        sys.exit(1)

    if not api.is_trusted_session:
        print("Session is not trusted. Requesting trust...")
        result = api.trust_session()
        print("Session trust result %s" % result)

        if not result:
            print("Failed to request trust. You will likely be prompted for the code again in the coming weeks")
elif api.requires_2sa:
    import click
    print ("Two-step authentication required. Your trusted devices are:")

    devices = api.trusted_devices
    for i, device in enumerate(devices):
        print ("  %s: %s" % (i, device.get('deviceName',
            "SMS to %s" % device.get('phoneNumber'))))

    device = click.prompt('Which device would you like to use?', default=0)
    device = devices[device]
    if not api.send_verification_code(device):
        print ("Failed to send verification code")
        sys.exit(1)

    code = click.prompt('Please enter validation code')
    if not api.validate_verification_code(device, code):
        print ("Failed to verify verification code")
        sys.exit(1)

Username:··········
Password:··········
Two-factor authentication required.
Enter the code you received of one of your approved devices: 476774


Code validation result: True


In [ ]:
drive_file = api.drive['Shortcuts']['Finances']['Expenses.numbers']

In [ ]:
with drive_file.open(stream=True) as response:
  with open(drive_file.name, 'wb') as file_out:
    copyfileobj(response.raw, file_out)

In [ ]:
doc = Document('Expenses.numbers')
sheets = doc.sheets()
tables = sheets[0].tables()
data = tables[0].rows(values_only=True)
df = pd.DataFrame(data)

In [ ]:
df.drop(index=df.index[0], axis=0, inplace=True)
df.rename(columns={0: 'Item', 1: 'DateTime'}, inplace=True)

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
df['Date'] = df['DateTime'].dt.date
df['Time'] = df['DateTime'].dt.time
df['Price'] = df['Item'].str.extract(r'(\d+\.?\d+)')
df['Item Bought'] = df['Item'].str.extract(r'\b(?!Lazada|Shopee)([\w?\s]+)(?<![ \d+])')

In [ ]:
df

,Item,DateTime,Date,Time,Price,Item Bought
1,Service fee 10,2022-12-30 22:48:00+00:00,2022-12-30,22:48:00,10,Service fee
2,Tip 20,2022-12-30 15:26:00+00:00,2022-12-30,15:26:00,20,Tip
3,Dinner 380,2022-12-29 22:46:00+00:00,2022-12-29,22:46:00,380,Dinner
4,Tricycle Fare 20,2022-12-30 11:01:51+00:00,2022-12-30,11:01:51,20,Tricycle Fare
5,Auntie Anne’s 220,2022-12-30 18:35:31+00:00,2022-12-30,18:35:31,220,Auntie Anne
...,...,...,...,...,...,...
95,None,NaT,NaN,NaN,NaN,NaN
96,None,NaT,NaN,NaN,NaN,NaN
97,None,NaT,NaN,NaN,NaN,NaN
98,None,NaT,NaN,NaN,NaN,NaN


In [ ]:
#List for Categories

food = ['McDonalds', 'Jollibee', 'Wildflour', 'Starbucks', 'Kangkong Chips', 'Eggs', 'Ramen',
        'Breakfast', 'Lunch', 'Dinner', 'Order Ready', 'Coffee', 'Bread Talk', 'Bread', 'Merienda', 'Doughnut'
        'Ice Cream', 'Doughnut', 'Lunch Thai', 'Frozen yoghurt', 'Samgyupsal']
travel = ['Parking', 'RFID', 'Gas', 'Gasoline', 'Driver', 'Toll', 'Jeepney', 'Jeepney Fare', 'Toll Fee', 'Tricycle']
grocery = ['Grocery', 'Korean Grocery']
in_app = ['Apple', 'Google', 'Discord', 'Telegram', 'VPN']
entertainment = ['Cinema', 'Cards']

merged = food + travel + grocery + in_app + entertainment

In [ ]:
df['Category'] = np.nan
df['Item Bought'] = df['Item Bought'].str.replace(r'^[\s]+|[\s]+$', '', regex=True)

In [ ]:
df.loc[df['Item Bought'].isin(food), 'Category'] = 'Food'
df.loc[df['Item Bought'].isin(travel), 'Category'] = 'Travel'
df.loc[df['Item Bought'].isin(grocery), 'Category'] = 'Grocery'
df.loc[df['Item Bought'].isin(in_app), 'Category'] = 'In App Transactions'
df.loc[df['Item Bought'].isin(entertainment), 'Category'] = 'Entertainment'
df.loc[(~df['Item Bought'].isin(merged)) & (df['Item'].notna()), 'Category'] = 'Others'

In [ ]:
df.drop(columns='Item', inplace=True)
df.dropna(inplace=True)
df['Price'] = df['Price'].astype(float)
df['Payment Type'] = 'Cash'
df.rename(columns={'Item Bought': 'Item'}, inplace=True)

In [ ]:
final_df_cash = df[['Date', 'Price', 'Item', 'Category', 'Payment Type']]

In [ ]:
final_df_cash

,Date,Price,Item,Category,Payment Type
1,2022-12-30,10.0,Service fee,Others,Cash
2,2022-12-30,20.0,Tip,Others,Cash
3,2022-12-29,380.0,Dinner,Food,Cash
4,2022-12-30,20.0,Tricycle Fare,Others,Cash
5,2022-12-30,220.0,Auntie Anne,Others,Cash
...,...,...,...,...,...
74,2022-01-11,50.0,Parking,Travel,Cash
75,2022-01-07,350.0,Driver,Travel,Cash
76,2022-01-07,331.0,McDonalds,Food,Cash
77,2022-01-04,60.0,Eggs,Food,Cash


#Merge all Tables and Finalize

In [ ]:
final_df = pd.concat([final_df_sbc, final_df_ub, final_df_ew, final_df_cash])
final_df.sort_values(by='Date', inplace=True)
final_df['Year'] = final_df['Date'].apply(lambda x: x.strftime('%Y'))
final_df['Month'] = final_df['Date'].apply(lambda x: x.strftime('%m'))
final_df['Day'] = final_df['Date'].apply(lambda x: x.strftime('%Y'))
final_df['Date'] = final_df['Date'].astype(str)

In [ ]:
final_df

,Date,Price,Item,Category,Payment Type,Year,Month,Day
0,2021-11-21,149.0,APPLE.COM/BILL,In-App Purchase,Security Bank Platinum Credit Card,2021,11,2021
1,2021-11-21,399.0,APPLE.COM/BILL,In-App Purchase,Security Bank Platinum Credit Card,2021,11,2021
2,2021-11-21,199.0,APPLE.COM/BILL,In-App Purchase,Security Bank Platinum Credit Card,2021,11,2021
3,2021-11-21,699.0,APPLE.COM/BILL,In-App Purchase,Security Bank Platinum Credit Card,2021,11,2021
4,2021-11-22,519.0,DISCORD* NITROMONTHLY,In-App Purchase,Security Bank Platinum Credit Card,2021,11,2021
...,...,...,...,...,...,...,...,...
2,2022-12-30,20.0,Tip,Others,Cash,2022,12,2022
1,2022-12-30,10.0,Service fee,Others,Cash,2022,12,2022
5,2022-12-30,220.0,Auntie Anne,Others,Cash,2022,12,2022
6,2022-12-30,12.0,Jeepney Fare,Travel,Cash,2022,12,2022


#Export as Google Sheets, Then Open Looker Studio to Make/Edit/Modify Dashboards

In [ ]:
workbook = gc.open_by_url('open_google_sheets_for_merged_tables')

In [ ]:
worksheet = workbook.sheet1

In [ ]:
worksheet.update([final_df.columns.values.tolist()] + final_df.values.tolist())

{'spreadsheetId': '1FrtvRLAxoEpZh_CYzyE6iP_pAmk_vHPldd-08sZ0558',
 'updatedRange': "'Sheet 1'!A1:H580",
 'updatedRows': 580,
 'updatedColumns': 8,
 'updatedCells': 4640}